**Run the following two cells before you begin.**

In [1]:
%autosave 10

Autosaving every 10 seconds


In [2]:
import pandas as pd
import numpy as np

______________________________________________________________________
**First, import your data set and define the sigmoid function.**
<details>
    <summary>Hint:</summary>
    The definition of the sigmoid is $f(x) = \frac{1}{1 + e^{-X}}$.
</details>

In [3]:
# Import the data set
df = pd.read_csv(r"D:\Technocolabs\Mini Projects\Mini Project 2\DATASETS\cleaned_data.csv")

In [4]:
# Define the sigmoid function
d = []
def sigmoid(z):
    for i in z:
        p = (1 / (1 + np.exp(-i)))
        d.append(p)
    return d

**Now, create a train/test split (80/20) with `PAY_1` and `LIMIT_BAL` as features and `default payment next month` as values. Use a random state of 24.**

In [5]:
# Create a train/test split
X = df[["PAY_1" , "LIMIT_BAL"]]
y = df[['default payment next month']]
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(X , y , train_size = 0.8)

______________________________________________________________________
**Next, import LogisticRegression, with the default options, but set the solver to `'liblinear'`.**

In [6]:
from sklearn.linear_model import LogisticRegression
regressor = LogisticRegression(solver = 'liblinear')

______________________________________________________________________
**Now, train on the training data and obtain predicted classes, as well as class probabilities, using the testing data.**

In [7]:
# Fit the logistic regression model on training data
regressor.fit(x_train , y_train)

c:\users\vibhu gupta\appdata\local\programs\python\python38\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LogisticRegression(solver='liblinear')

In [8]:
# Make predictions using `.predict()`
predictions = regressor.predict(x_test)

In [9]:
# Find class probabilities using `.predict_proba()`
probs = regressor.predict_proba(x_test)

______________________________________________________________________
**Then, pull out the coefficients and intercept from the trained model and manually calculate predicted probabilities. You'll need to add a column of 1s to your features, to multiply by the intercept.**

In [10]:
# Add column of 1s to features
ones = np.ones(26664)
data = {"Ones" : ones , "PAY_1" : df.PAY_1 ,"LIMIT_BAL" : df.LIMIT_BAL}
X = pd.DataFrame(data) 
x_trainm , x_testm , y_trainm , y_testm = train_test_split(X , y , train_size = 0.8)

In [11]:
# Get coefficients and intercepts from trained model
coeffs = regressor.coef_
intercept = regressor.intercept_

In [12]:
# Manually calculate predicted probabilities
m1 = coeffs[0][0]
m2 = coeffs[0][1]
a = x_testm.iloc[:,1]
b = x_testm.iloc[:,2]
c = x_testm.iloc[:,0]
z = m1*a + m2*b + c*intercept 
man_pred_probs = sigmoid(z)
all(man_pred_probs) == all(probs[:,1])

True

______________________________________________________________________
**Next, using a threshold of `0.5`, manually calculate predicted classes. Compare this to the class predictions output by scikit-learn.**

In [13]:
# Manually calculate predicted classes
threshold = 0.5
man_pred = []
for i in man_pred_probs:
    if(i > threshold):
        man_pred.append(1)
    else:
        man_pred.append(0)
man_pred.count(0)

5333

In [14]:
# Compare to scikit-learn's predicted classes
all(man_pred) == all(predictions)

True

# ______________________________________________________________________
**Finally, calculate ROC AUC using both scikit-learn's predicted probabilities, and your manually predicted probabilities, and compare.**

In [15]:
# Use scikit-learn's predicted probabilities to calculate ROC AUC
from sklearn.metrics import roc_auc_score
score = roc_auc_score(y_test , probs[:,1])
score

0.6374971093653305

In [16]:
# Use manually calculated predicted probabilities to calculate ROC AUC
man_score = roc_auc_score(y_testm , man_pred_probs)
man_score

0.6173503073081725